In [3]:
import urllib.request as url
import pandas as pd
from geopy.distance import vincenty
import webbrowser
import json
import time
import subprocess
import os
from geopy.geocoders import Nominatim
geolocator = Nominatim()
LAT = 40.7829
LON = 73.9654
global source
source = (LAT,LON)

In [4]:
table_data = pd.read_csv('open_table_data_final.csv',encoding = "ISO-8859-1")
len(table_data)


595

In [5]:
table_data[:5]["lat"]


0    40.745962
1    40.745394
2    40.751123
3    40.748476
4    40.748698
Name: lat, dtype: float64

In [6]:
table_data[:5]["lng"]

0   -73.988622
1   -73.989588
2   -74.002069
3   -74.003339
4   -74.003228
Name: lng, dtype: float64

Call the below function when you data needs to merged and data is manually cleaned for required cuisines

In [7]:
def createMergedDataSet():
    pd1 = pd.read_csv('open_table_data_final.csv',encoding = "ISO-8859-1")
    pd2 = pd.read_csv('metaDataRestaurant.csv',encoding = "ISO-8859-1")
    merged = pd1.merge(pd2,how='outer',on='id')
    merged.to_csv('data_set_merged.csv',encoding = "ISO-8859-1")

In [8]:
def calculateDistance(table_data,source):
    distance = []
    #print(source)
    #table_data1 = pd.DataFrame()
    for i in range(len(table_data)):
        lat_dest = table_data.at[i,"lat"]
        lng_dest = table_data.at[i,"lng"]
        dest = (lat_dest,lng_dest)
        distance.append(vincenty(source, dest).miles)
    table_data = table_data.assign(dist = distance)
    #print(table_data.sort_values(by=['price','dist'], ascending=[True,True])[1:5])
    return table_data


In [9]:
#using geopy displacement
def getLatAndLng(place):
    location = geolocator.geocode(place)
    print((location.latitude, location.longitude))
    return [location.latitude,location.longitude]

In [10]:
#using osrm matrix distance
def calculateDistanceOSRM(table_data,source):
    #print(table_data.head())
    distance = []
    table_temp = pd.DataFrame()
    #data = json.load(url.urlopen("http://router.project-osrm.org/table/v1/drive/-73.987330,40.748711;-74.0134,40.7127?sources=1"))
    #for i in range(5):
    for i in range(len(table_data)):
        #lat_dest = table_data.at[i,"lat"]
        lat_dest = table_data["lat"][i]
        #lng_dest = table_data.at[i,"lng"]
        lng_dest = table_data["lng"][i]
        json_data = url.urlopen("http://router.project-osrm.org/table/v1/drive/"+str(source[1])+","+str(source[0])+";"+str(lng_dest)+","+str(lat_dest)+"?sources=1")
        data = json.load(json_data)
        calc_distance = round((data['durations'][0][0])*0.000621371,4)
        distance.append(calc_distance)
    
    #table_temp = table_data[0:5]
    table_temp = table_data
    table_temp = table_temp.assign(dist = distance)
    #table_data = table_data.assign(dist = distance)
    #print(table_data.sort_values(by=['price','dist'], ascending=[True,True])[1:5])
    #return table_data
    return table_temp

In [11]:
def searchForRestaurant(query,table_data):
    table_data_dist_osrm = pd.DataFrame()
    print("SearchForRestaurant",len(table_data))
    if("near me" in query or "nearme" in query):
        LAT = 40.7829
        LON = 73.9654
        source = (LAT,LON)
    else:
        place = query.split("near")
        loc = getLatAndLng(place)
        LAT = loc[0]
        LON = loc[1]
        source = (LAT,LON)
        table_data_dist = pd.DataFrame()
        #table_data_dist = calculateDistance(table_data,source)
        tabel_data_dist_osrm = calculateDistanceOSRM(table_data,source)
        #print(type(table_data_dist_osrm))
    return (table_data_dist,tabel_data_dist_osrm,LAT,LON)

In [12]:
def openMap(src_LAT,src_LON,des_LAT,des_LON):
    webbrowser.open("https://www.openstreetmap.org/directions?engine=graphhopper_car&route="+str(src_LAT)+"%2C"+str(src_LON)+"%3B"+str(des_LAT)+"%2C"+str(des_LON),new=2)
    time.sleep(1)

In [13]:
def call_recommender():
    os.remove('E:\\Google Drive\\Fall 17 Sem 2\\620 Data Visualization\\Project\\Smart_Navigation\\Open_Table\\success')
    subprocess.call('run_recommender.bat')
    #while(not(os.path.exists('E:\\Google Drive\\Fall 17 Sem 2\\620 Data Visualization\\Project\\Smart_Navigation\\Open_Table\\success'))):
        #None

In [14]:
def defactorizeCuisine():
    recommended_data = pd.read_csv('recommendation.csv',encoding = "ISO-8859-1")
    #print(recommended_data.head())
    factor = ['American', ' Italian', 'Steakhouse', 'French', 'Indian', 'Mexican', 'Japanese','Chinese',  'Spanish', 'Pizzeria','Burgers','Korean', 'Vegetarian', 'Dessert'] 
    for i in range(0,len(factor)):
        #print(factor[i])
        recommended_data['Cuisine'] = recommended_data['Cuisine'].replace(i+1, factor[i], regex=True)
    recommended_data.to_csv('recommendation.csv',encoding = "ISO-8859-1")

In [15]:
def cleanMergedData():
    merged_data = pd.read_csv('data_set_merged-cleaned.csv',encoding = "ISO-8859-1")
    pd1 = pd.read_csv('recommendation.csv',encoding = "ISO-8859-1")
    recommended_cuisine = pd1['Cuisine']
    for i in range(len(merged_data)):
        cuisine = merged_data.at[i,'cuisines']
        for j in range(len(pd1)):
            if pd1.at[j,'Cuisine'] in cuisine:
                merged_data.at[i,'cuisines'] = pd1.at[j,'Cuisine']
                #print(pd1.at[j,'Cuisine'],cuisine)
    merged_data.to_csv('data_set_merged_cleaned.csv',encoding = "ISO-8859-1")            

In [17]:
table_data_dist = pd.DataFrame()
tabel_data_dist_osrm = pd.DataFrame()
temp = pd.DataFrame()
call_recommender()
createMergedDataSet()
defactorizeCuisine()
cleanMergedData()
pd1 = pd.read_csv('data_set_merged_cleaned.csv',encoding = "ISO-8859-1")
pd2 = pd.read_csv('recommendation.csv',encoding = "ISO-8859-1")
cuisine_value = pd2['Cuisine']
data = pd1[(pd1['cuisines']==cuisine_value[0]) | (pd1['cuisines']==cuisine_value[1]) | (pd1['cuisines']==cuisine_value[2]) | (pd1['cuisines']==cuisine_value[3]) | (pd1['cuisines']==cuisine_value[4])]
data = data.merge(table_data,how='inner',on='id')
data = data.rename(index=str, columns = {"lat_y":"lat","lng_y":"lng","price_y":"price"})
print(len(data))
#print(data.head())
#print(data['lat'])
#pd2.head()

53


In [19]:
def searchForRestaurants():
    print("Enter the location where you want to satisfy your foodie desire:")
    table_data_dist,tabel_data_dist_osrm,src_LAT,src_LON = searchForRestaurant(input(),data)
    #print(type(table_data_dist))
    #table_data_dist.sort_values(by=['price','dist'], ascending=[True,True])[1:5]
    temp = tabel_data_dist_osrm.sort_values(by=['price','dist'], ascending=[True,True])[1:6]
    print(temp['name_x'])
    for i in range(5):
        openMap(src_LAT,src_LON,temp.iloc[i]["lat"],temp.iloc[i]["lng"])

In [29]:
searchForRestaurants()

Enter the location where you want to satisfy your foodie desire:
near bryant park
SearchForRestaurant 53
(40.75374985, -73.9835672012434)
29                 Thai Select
50                    Il Forno
1          Stella 34 Trattoria
13    Trattoria Zero Otto Nove
30                      Iguana
Name: name_x, dtype: object


In [ ]:
#calculateDistance()
#table_data_dist
#table_data[:5]
#table_data.sort_values(by=['price','dist'], ascending=[True,True])[1:5]
print(temp.iloc[0]["lat"])
print(temp.iloc[0]["lng"])

In [ ]:
#table_data_dist,tabel_data_dist_osrm,src_LAT,src_LON = searchForRestaurant(input(),data)
#len(data)
table_data_dist = pd.DataFrame()
tabel_data_dist_osrm = pd.DataFrame()
temp = pd.DataFrame()
call_recommender()
createMergedDataSet()
defactorizeCuisine()
cleanMergedData()

In [27]:
geolocator.geocode("Wall Street")

Location(Wall Street, Southbridge Towers, Manhattan Community Board 1, New York County, NYC, New York, 10005, United States of America, (40.7048017, -74.0069496, 0.0))